# <center> ETL (Extract , Transform and Load) </center>
<center>En este Jupyter Notebook realizaremos el proceso de ETL a los 3 datasets que nos brindaron para este proyecto, y así, poder preparar los datos para responder las consultas y realizar los modelos de Machine Learning.</center>

## Extraccion de los datos

Notamos que los datasets que nos brindaron, estan comprimidos en formato GZIP, asi que procedimos a descomprimirlo para así poder tener los datasets en formato JSON y podeer leerlos correctamente con las librerias adecuadas.

### Importacion de librerias

In [23]:
import pandas as pd
import numpy as np
import Utilities as u
import json
import ast


import warnings
warnings.filterwarnings("ignore")

### Definimos la ruta de los dataset

In [2]:
dataGames = './dataset/Original/Comprimido/output_steam_games.json'
dataUserItems = './dataset/Original/Comprimido/australian_users_items.json'
dataUserReviews = './dataset/Original/Comprimido/australian_user_reviews.json'

### Leemos los datos y los convertimos en un DataFrame

#### Dataset de *output_steam_games.json*

In [3]:
data_games = []   #--> Creamos una lista vacía para luego almacenar los objetos JSON.

with open(dataGames, encoding= 'utf-8') as file:            #--> Abrimos el archivo y le damos el alias como 'file'    
    for line in file.readlines():       #--> Iteramos a través de las líneas del archivo uno por uno
        json_obj = json.loads(line)     #--> Convertimos esas lineas del archivo en un objeto JSON
        data_games.append(json_obj)     #--> Agregamos el objeto JSON a la lista   

In [ ]:
steam_games = pd.DataFrame(data_games)     #--> Lo convertimos en un DataFrame   

#### Dataset de *australian_users_items.json* 

En este caso lo procesamos de manera diferente, ya que el archivo tiene comillas simples, y si lo convertimos en objeto JSON, nos dara error, ya que en los objetos JSON las comillas simples no son válidas.

In [3]:
data_items = []   

with open(dataUserItems, encoding= 'utf-8') as file:
    for line in file.readlines():
        data_items.append(ast.literal_eval(line))   #--> Evaluamos y convertimos cada linea en un diccionario.    

In [ ]:
user_items = pd.DataFrame(data_items)   

#### Dataset de *australian_user_reviews.json*

In [8]:
data_reviews = []   

with open(dataUserReviews, encoding= 'utf-8') as file:
    for line in file.readlines():
        data_reviews.append(ast.literal_eval(line))         

In [ ]:
user_reviews = pd.DataFrame(data_reviews) 

### Guardamos los DataFrames en archivos CSV

Una vez que obtenemos un DataFrame por cada dataset, procedemos a guardarlo en un archivo con formato CSV para poder cargarlo de manera más rápida.


#### DataFrame *df_games*

In [25]:
steam_games.to_csv('./dataset/Original/CSV/steam_games.csv', encoding= 'utf-8', index=False)  # El argumento 'index=False' evita que se guarde el índice del DataFrame 

#### DataFrame *df_items*

In [29]:
user_items.to_csv('./dataset/Original/CSV/users_items.csv', encoding= 'utf-8', index=False)

#### DataFrame *df_reviews*

In [30]:
user_reviews.to_csv('./dataset/Original/CSV/user_reviews.csv', encoding= 'utf-8', index=False)

## Transformacion de los datos

### Lectura de los datasets

Leemos los archivos CSV guardados anteriormente

In [5]:
steam_games = pd.read_csv('./dataset/Original/CSV/steam_games.csv')

user_items = pd.read_csv('./dataset/Original/CSV/users_items.csv' )

user_reviews = pd.read_csv('./dataset/Original/CSV/user_reviews.csv')

### Información general de los DataFrames

#### DataFrame de *steam_games*

In [15]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24064 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  object 
 12  id              32133 non-null  float64
 13  metascore       2607 non-null   float64
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  float64
 17  items           88310 non-nul

Oberservamos que hay bastantes valores nulos en cada columna, veamos un poco mas en detalle:

In [17]:
u.porcentaje_nulos(steam_games)

La columna publisher tiene un 80.0% de valores nulos.
La columna genres tiene un 76.0% de valores nulos.
La columna app_name tiene un 73.3% de valores nulos.
La columna title tiene un 75.0% de valores nulos.
La columna url tiene un 73.3% de valores nulos.
La columna release_date tiene un 75.0% de valores nulos.
La columna tags tiene un 73.5% de valores nulos.
La columna reviews_url tiene un 73.3% de valores nulos.
La columna discount_price tiene un 99.8% de valores nulos.
La columna specs tiene un 73.9% de valores nulos.
La columna price tiene un 74.5% de valores nulos.
La columna early_access tiene un 73.3% de valores nulos.
La columna id tiene un 73.3% de valores nulos.
La columna metascore tiene un 97.8% de valores nulos.
La columna developer tiene un 76.1% de valores nulos.
La columna user_id tiene un 26.7% de valores nulos.
La columna steam_id tiene un 26.7% de valores nulos.
La columna items tiene un 26.7% de valores nulos.
La columna items_count tiene un 26.7% de valores nulos.


In [18]:
steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evcentric,7.656120e+16,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riot-Punch,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doctr,7.656120e+16,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0


#### DataFrame de *user_items*

In [19]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [20]:
user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


#### DataFrame de *user_reviews*

In [21]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [22]:
user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


### Desanidar registros

Notamos que hay registros anidados en los 3 DataFrames, vamos a desanidarlo para poder tener limpio los datos. Veamos que columnas tienen registros anidados.

#### Vamos a ver que columnas tienen registros anidados en el DataFrame *'df_games'*

In [24]:
u.registros_anidados(steam_games)

['items']

#### Vamos a ver que columnas tienen registros anidados en el DataFrame *'df_items'*

In [25]:
u.registros_anidados(user_items)

['items']

#### Vamos a ver que columnas tienen registros anidados en el DataFrame *'df_reviews'*

In [26]:
u.registros_anidados(user_reviews)

['reviews']

Concluimos con que solo una columna en cada DataFrame, tiene registros anidados. <br>
* En el caso del DataFrame *'steam_games'* : **'items'**
* En el caso del DataFrame *'user_items'* : **'items'**
* En el caso del DataFrame *'user_reviews'* : **'reviews'**

Veamos los registros que tienen estas columnas

In [27]:
steam_games['items'][0]

'[{\'item_id\': \'10\', \'item_name\': \'Counter-Strike\', \'playtime_forever\': 6, \'playtime_2weeks\': 0}, {\'item_id\': \'20\', \'item_name\': \'Team Fortress Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'30\', \'item_name\': \'Day of Defeat\', \'playtime_forever\': 7, \'playtime_2weeks\': 0}, {\'item_id\': \'40\', \'item_name\': \'Deathmatch Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'50\', \'item_name\': \'Half-Life: Opposing Force\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'60\', \'item_name\': \'Ricochet\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'70\', \'item_name\': \'Half-Life\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'130\', \'item_name\': \'Half-Life: Blue Shift\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'300\', \'item_name\': \'Day of Defeat: Source\', \'playtime_forever\': 4733, \'playtime_2weeks\': 0}, {\'ite

In [28]:
user_items['items'][0]

'[{\'item_id\': \'10\', \'item_name\': \'Counter-Strike\', \'playtime_forever\': 6, \'playtime_2weeks\': 0}, {\'item_id\': \'20\', \'item_name\': \'Team Fortress Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'30\', \'item_name\': \'Day of Defeat\', \'playtime_forever\': 7, \'playtime_2weeks\': 0}, {\'item_id\': \'40\', \'item_name\': \'Deathmatch Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'50\', \'item_name\': \'Half-Life: Opposing Force\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'60\', \'item_name\': \'Ricochet\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'70\', \'item_name\': \'Half-Life\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'130\', \'item_name\': \'Half-Life: Blue Shift\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'300\', \'item_name\': \'Day of Defeat: Source\', \'playtime_forever\': 4733, \'playtime_2weeks\': 0}, {\'ite

In [29]:
user_reviews['reviews'][0]

'[{\'funny\': \'\', \'posted\': \'Posted November 5, 2011.\', \'last_edited\': \'\', \'item_id\': \'1250\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.\'}, {\'funny\': \'\', \'posted\': \'Posted July 15, 2011.\', \'last_edited\': \'\', \'item_id\': \'22200\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': "It\'s unique and worth a playthrough."}, {\'funny\': \'\', \'posted\': \'Posted April 21, 2011.\', \'last_edited\': \'\', \'item_id\': \'43110\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!\'}

Comprobamos el tipo de dato de los registros:

In [5]:
type(steam_games['items'][0])

str

In [32]:
type(user_items['items'][0])

str

In [34]:
type(user_reviews['reviews'][0])

str

Notamos que hay un problema en estos registros, y es por la forma en la que se guardan las columnas que tienen listas o diccionarios, ya que Pandas tiende a serializar estos objetos en una cadena y los almacena asi.,<br>
Entonces vamos a modificar estos registros para que vuelvan a su forma original.

In [5]:
steam_games['items'] = steam_games['items'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

<dl>
  <dt>Explicacion del código:</dt>
  <dd>Esta función lambda evalua el valor de cada elemento de la columna como una expresión literal, en este caso, convierte cada elemento en una lista. y solo si el elemento no esta vacio. Si esta vacio, entonces lo define como 'nan'</dd>
</dl>

In [11]:
user_items['items'] = user_items['items'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

In [12]:
user_reviews['reviews'] = user_reviews['reviews'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

Corroboramos los cambios

In [7]:
type(steam_games['items'][0])

list

In [39]:
type(user_items['items'][0])

list

In [40]:
type(user_reviews['reviews'][0])

list

Bien, ahora podemos comenzar con el proceso de desanidación de los registros.

#### Columna ***'Items'*** 

Notamos que tanto el DataFrame 'steam_games' como el de 'user_items' tienen el mismo nombre de columna en sus registros anidados.
Asi que veamos si son los mismos registros.

* DataFrame **'steam_games'**

In [13]:
steam_games['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

* DataFrame **'user_items'**

In [45]:
user_items['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

A simple vista pareciera ser los mismos registros... <br>
Para asegurarnos, podemos corroborar estos registros con la informacion que nos brinda el DataFrame 'user_items' en la columna 'items_count', pero esto lo haremos luego de desanidar estos registros.

Para desanidar, vamos a utilizar la lista con los registros que obtuvimos al leer el archivo JSON 'output_steam_games'. Y aprovecharemos para incluir algunas columnas que nos serviran para hacer las funciones. 
Como estos datos son de **items**, agregaremos la columna de 'user_id' para saber **qué** items tiene cada usuario y ademas la columna de 'genres' que nos servira para las funciones.

In [89]:
items = pd.json_normalize(data_games, record_path='items' , meta=['user_id'])

En el argumento 'record_path' indicamos que columna normalizar.<br>
En el argumento 'meta' especificamos la columna 'user_id' para incluirlo en el DataFrame.

In [90]:
items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


Veamos la dimension del DataFrame

In [91]:
items.shape

(5153209, 5)

Vemos si tiene filas duplicadas

In [93]:
u.filas_duplicadas(items)

'El Dataframe tiene 59104 filas duplicadas'

Procedemos a eliminar las filas duplicadas, manteniendo la primera ocurrencia de cada fila duplicada

In [94]:
items.drop_duplicates(keep = 'first', inplace = True)

Corroboramos

In [95]:
u.filas_duplicadas(items)

'El DataFrame no tiene filas duplicadas'

Veamos la cantidad de registros unicos en la columna *'user_id'* en el DataFrame **'user_items'**

In [97]:
u.registros_unicos(user_items,'user_id')

'La columna user_id tiene 87626 registros unicos'

Ahora veamos cuantos registros unicos en la columna *'user_id'* hay en el DataFrame **'items'**

In [98]:
u.registros_unicos(items,'user_id')

'La columna user_id tiene 70912 registros unicos'

Notamos que es diferente la cantidad de registros unicos de la columna *'user_id'* en cada DataFrame. Pero puede ser que sea porque hay usuarios donde no tienen items.<p>
Para corroborar esto, veamos cuantos usuarios no tienen items

In [99]:
user_items.user_id[user_items['items_count'] == 0].nunique()

16714

Ahora corroboremos la informacion de la cantidad de items que tiene cada usuario que nos brinda el DataFrame **user_items**

In [100]:
def corroborar_items(user):
    cantidad_items = (items['user_id'] == user).sum()
    cantidad_user = user_items.loc[user_items['user_id'] == user]['items_count'].iloc[0]

    if cantidad_items == cantidad_user:
        return 'Los registros concuerdan en ambos DataFrames ("user_items", "items")'
    else: 
        return 'Los registros no concuerdan'

In [101]:
corroborar_items('76561198326700687')

'Los registros concuerdan en ambos DataFrames ("user_items", "items")'

Bien, confirmamos que los registros que sacamos de los datos anidados estan bien. Ya que si restamos la cantidad de usuarios que hay en el DataFrame **'user_items'** con la cantidad que hay en **'items'**, nos devolvera la misma cantidad de usuarios que no tienen items.<p>
``` 87626 - 70912 = 16714 ```

Y ademas, la cantidad de items que tenemos por usuario, es igual a la informacion que nos brinda el DataFrame **user_items**.<p>
Entonces no hace falta desanidar los registros de la columna *'items'* del DataFrame **user_items**. Asi que procedemos a eliminar esa columna.

In [102]:
user_items.drop(columns = 'items', inplace = True)

Bien, ahora agregamos la columna *'genres'* al DataFrame de **items**.

In [6]:
genero = steam_games[['id','genres']]

In [7]:
genero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32133 non-null  float64
 1   genres  28852 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.8+ MB


In [8]:
genero.dropna(how='all', inplace=True) # Eliminamos las filas que tengan valores nulos en todas las columnas.

In [9]:
genero.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32134 entries, 88310 to 120444
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32133 non-null  float64
 1   genres  28852 non-null  object 
dtypes: float64(1), object(1)
memory usage: 753.1+ KB


Verificamos el tipo de dato de la columna *'item_id'* para poder unir ambos DataFrames, ya que el tipo de dato de la columna *'id'* es de tipo *float64*.

In [10]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   user_id           object
 3   playtime_forever  int64 
 4   playtime_2weeks   int64 
dtypes: int64(3), object(2)
memory usage: 194.3+ MB


Modificamos el tipo de dato de la columna 'itemd_id'

In [11]:
items['item_id'] = items['item_id'].astype(float)

Corroboramos

In [12]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           float64
 1   item_name         object 
 2   user_id           object 
 3   playtime_forever  int64  
 4   playtime_2weeks   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 194.3+ MB


Unimos ambos DataFrames segun el valor del 'item_id'

In [13]:
items = items.merge(genero[['id', 'genres']], left_on='item_id', right_on='id', how='left')

In [14]:
items.head()

,item_id,item_name,user_id,playtime_forever,playtime_2weeks,id,genres
0,10.0,Counter-Strike,76561197970982479,6,0,10.0,['Action']
1,20.0,Team Fortress Classic,76561197970982479,0,0,20.0,['Action']
2,30.0,Day of Defeat,76561197970982479,7,0,30.0,['Action']
3,40.0,Deathmatch Classic,76561197970982479,0,0,40.0,['Action']
4,50.0,Half-Life: Opposing Force,76561197970982479,0,0,50.0,['Action']


Como la columna 'id' es igual a la columna 'item_id', la eliminaremos.

In [15]:
items.drop(columns = 'id', inplace = True)

In [16]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           float64
 1   item_name         object 
 2   user_id           object 
 3   playtime_forever  int64  
 4   playtime_2weeks   int64  
 5   genres            object 
dtypes: float64(1), int64(2), object(3)
memory usage: 233.2+ MB


Modificaremos el tipo de dato de la columna *'item_id'*

In [17]:
items['item_id'] = items['item_id'].astype(int)

In [18]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int32 
 1   item_name         object
 2   user_id           object
 3   playtime_forever  int64 
 4   playtime_2weeks   int64 
 5   genres            object
dtypes: int32(1), int64(2), object(3)
memory usage: 213.8+ MB


Notamos que la columna *'genres'* tiene valores en formato lista, y contiene mas de 1 solo valor algunas filas, entonces vamos a descomponer esas listas para tener filas individuales y poder simplificar el analisis.

In [ ]:
items['genres'] = items['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

Utilizamos el método **explode()** para descomponer las listas.

In [ ]:
items = items.explode('genres')

Verificamos

In [42]:
items.head()

,item_id,item_name,user_id,genres,playtime_forever,playtime_2weeks
0,10,Counter-Strike,76561197970982479,Action,6,0
1,20,Team Fortress Classic,76561197970982479,Action,0,0
2,30,Day of Defeat,76561197970982479,Action,7,0
3,40,Deathmatch Classic,76561197970982479,Action,0,0
4,50,Half-Life: Opposing Force,76561197970982479,Action,0,0


Veamos los registros únicos de la columna *'genres'*

In [3]:
items.genres.unique()

array(['Action', 'Strategy', nan, 'RPG', 'Indie', 'Casual', 'Simulation',
       'Adventure', 'Racing', 'Sports', 'Early Access', 'Free to Play',
       'Massively Multiplayer', 'Utilities', 'Animation &amp; Modeling',
       'Design &amp; Illustration', 'Education', 'Web Publishing',
       'Video Production', 'Audio Production', 'Software Training',
       'Photo Editing'], dtype=object)

Notamos que hay un error al identificar el caracter *'&'*, asi que lo modificaremos por un **'and'**

In [4]:
items['genres'] = items['genres'].str.replace('&amp;', 'and')

Corroboramos

In [5]:
items.genres.unique()

array(['Action', 'Strategy', nan, 'RPG', 'Indie', 'Casual', 'Simulation',
       'Adventure', 'Racing', 'Sports', 'Early Access', 'Free to Play',
       'Massively Multiplayer', 'Utilities', 'Animation and Modeling',
       'Design and Illustration', 'Education', 'Web Publishing',
       'Video Production', 'Audio Production', 'Software Training',
       'Photo Editing'], dtype=object)

Guardamos el DataFrames de Items desanidados, pero antes, ordeamos las columnas.

In [19]:
items = items[['item_id','item_name','user_id','genres','playtime_forever','playtime_2weeks']]

In [43]:
items.head(3)

,item_id,item_name,user_id,genres,playtime_forever,playtime_2weeks
0,10,Counter-Strike,76561197970982479,Action,6,0
1,20,Team Fortress Classic,76561197970982479,Action,0,0
2,30,Day of Defeat,76561197970982479,Action,7,0


In [7]:
items.to_csv('./dataset/Modificados/items.csv', encoding= 'utf-8', index=False)

#### Columna ***'Items'*** del DataFrame 'user_items'

Notamos que el DataFrame *'user_items'* contiene mas que nada, informacion del usuario y la columna Items con registros anidados, pero como anteriormente ya desanidamos estos datos, procedemos a eliminar esa columna y renombrar el DataFrame a *'users'* para dar a entender que se refiere a la informacion de usuarios.

In [35]:
users = user_items

In [36]:
users.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Eliminamos la columna *'items'*

In [37]:
users.drop(columns = 'items',inplace = True)

In [38]:
users.head()

,user_id,items_count,steam_id,user_url
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr


Verificamos duplicados

In [30]:
u.filas_duplicadas(users)

'El Dataframe tiene 684 filas duplicadas'

Eliminamos los duplicados

In [31]:
users.drop_duplicates(inplace = True)

Corroboramos

In [32]:
u.filas_duplicadas(users)

'El DataFrame no tiene filas duplicadas'

Ahora procedemos a guardar este DataFrame como *'users'*

In [34]:
users.to_csv('./dataset/Modificados/users.csv', encoding = 'utf-8', index=False)

#### Columna ***'Reviews'*** del DataFrame 'df_reviews'

In [40]:
reviews = pd.json_normalize(data_reviews, record_path=['reviews'] , meta=['user_id'])

En el argumento 'meta' especificamos la columna 'user_id' para incluirlo en el DatFrame.

In [42]:
reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


Echemos un vistazo a la informacion general de este DataFrame.

In [49]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   item_id      59305 non-null  object
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   funny        59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   helpful      59305 non-null  object
dtypes: bool(1), object(7)
memory usage: 3.2+ MB


Veamos si tiene duplicados

In [43]:
u.filas_duplicadas(reviews)

'El Dataframe tiene 874 filas duplicadas'

Eliminamos las filas duplicadas.

In [44]:
reviews.drop_duplicates(keep = 'first', inplace = True)

Corroboramos

In [45]:
u.filas_duplicadas(reviews)

'El DataFrame no tiene filas duplicadas'

Bien, ahora guardaremos este DataFrame, definiendolo como el dataset de reviews único. Ya que la informacion que nos brinda es practicamente la misma que en el **'df_reviews'**, solo que no incluimos la columna de ***'user_url'***. <p>
Pero antes, ordenamos las columnas.

In [46]:
reviews = reviews[['user_id','item_id', 'posted', 'last_edited', 'funny', 'recommend', 'review','helpful']]

In [69]:
reviews.to_csv('./dataset/Modificado/reviews.csv', encoding= 'utf-8', index=False)

#### Registros de Items en el DataFrame 'steam_games'

Como anteriormente ya separamos los datos de *items* y de *users*, procederemos a eliminar estas columnas del DataFrame **steam_games** para poder achicar la dimension del mismo.

In [138]:
steam_games.shape[1]

13

Eliminamos las columnas que no nos serviran.

In [64]:
steam_games.drop(columns=['user_id','steam_id','items','items_count'],inplace=True)

In [137]:
steam_games.shape[1]

13

### Modificación de los Datos

Ahora que ya tenemos los DataFrames desanidados, procederemos a realizar algunas modificaciones en estos datos.

#### DataFrame *'steam_games'*
En este DataFrame, vamos a extraer el anio de la columna *'release_date'* e imputar con la url que nos brinda la informacion del juego.

In [188]:
steam_games.release_date.unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

Vemos que el anio esta representado con 4 digitos, asi que vamos a extraerlos en una nueva columna.

In [205]:
steam_games['release_year'] = steam_games['release_date'].str.extract(r'(\b\d{4}\b)')

In [206]:
steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro,2018
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL,2018
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com,2017
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN


In [219]:
steam_games[['release_date','release_year']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   release_date  30068 non-null  object
 1   release_year  29964 non-null  object
dtypes: object(2)
memory usage: 502.2+ KB


Vemos que no hay la misma cantidad de registros que hay en la columna *'release_date'*, investigemos un poco

In [197]:
steam_games.loc[(steam_games.release_year.isna()) & (steam_games.release_date.notna())]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year
10,Qucheza,"['Action', 'Indie', 'Simulation', 'Early Access']",Uncanny Islands,Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,Soon..,"['Early Access', 'Action', 'Indie', 'Simulatio...",http://steamcommunity.com/app/768570/reviews/?...,['Single-player'],NaN,True,768570.0,Qucheza,NaN
44,zha7,"['Adventure', 'Free to Play', 'RPG', 'Early Ac...",Idle Adventure,Idle Adventure,http://store.steampowered.com/app/714020/Idle_...,Beta测试已开启,"['Early Access', 'Free to Play', 'Adventure', ...",http://steamcommunity.com/app/714020/reviews/?...,"['MMO', 'Steam Achievements']",NaN,True,714020.0,"zha7,Phaedruns,Yingke Wu",NaN
6737,Ghost Machine,"['Action', 'Indie']",Warhawks,Warhawks,http://store.steampowered.com/app/380130/Warha...,Coming Soon,"['Action', 'Indie', 'VR', 'Flight']",http://steamcommunity.com/app/380130/reviews/?...,"['Single-player', 'Multi-player', 'Steam Achie...",NaN,False,380130.0,Ghost Machine,NaN
7134,Strange Loop Games,"['Adventure', 'Indie', 'Simulation']",Eco,Eco,http://store.steampowered.com/app/382310/Eco/,Coming Soon,"['Survival', 'Adventure', 'Multiplayer', 'Simu...",http://steamcommunity.com/app/382310/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",NaN,False,382310.0,Strange Loop Games,NaN
7189,"Alterego Games, Digital Forest","['Adventure', 'Casual', 'Indie']",Woven,Woven,http://store.steampowered.com/app/504690/Woven/,TBA,"['Indie', 'Casual', 'Adventure', 'Singleplayer...",http://steamcommunity.com/app/504690/reviews/?...,['Single-player'],NaN,False,504690.0,"Alterego Games,Digital Forest",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31983,8Floor,"['Casual', 'Strategy']",Lost Artifacts,Lost Artifacts,http://store.steampowered.com/app/708150/Lost_...,Coming soon,"['Strategy', 'Casual']",http://steamcommunity.com/app/708150/reviews/?...,"['Single-player', 'Steam Achievements']",NaN,False,708150.0,Creobit,NaN
31989,Dagestan Technology,"['Action', 'Adventure', 'Indie']",Memento,Memento,http://store.steampowered.com/app/454940/Memento/,TBD,"['Indie', 'Action', 'Adventure', 'Horror']",http://steamcommunity.com/app/454940/reviews/?...,['Single-player'],NaN,False,454940.0,CRAZY SECTOR,NaN
32000,Maulidan Games,"['Adventure', 'Indie']",Spy of Deimos,Spy of Deimos,http://store.steampowered.com/app/689820/Spy_o...,Coming Soon,"['Indie', 'Adventure']",http://steamcommunity.com/app/689820/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",NaN,False,689820.0,Maulidan Games,NaN
32008,New Blood Interactive,"['Action', 'Indie']",DUSK,DUSK,http://store.steampowered.com/app/519860/DUSK/,SOON™,"['Action', 'FPS', 'Indie', 'Retro', 'Shooter',...",http://steamcommunity.com/app/519860/reviews/?...,"['Single-player', 'Online Multi-Player', 'Stea...",20.0,False,519860.0,David Szymanski,NaN


Notamos que hay valores donde no especifica el anio en la columna *'release_date'*. Asi que procedemos a hacer Web Scraping con las Urls que tenemos en la columna *'url'* para poder imputar estos registros

Aplicamos la funcion ***obtener_release_year*** a los registros vacios en la columna *'release_year'* y *'release_date'*

In [216]:
steam_games['release_year_2'] = steam_games.loc[(steam_games.release_year.isna()) & (steam_games.release_date.notna()), 'url'].apply(u.obtener_release_year)

In [220]:
steam_games[['release_date','release_year','release_year_2']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   release_date    30068 non-null  object
 1   release_year    29964 non-null  object
 2   release_year_2  90 non-null     object
dtypes: object(3)
memory usage: 753.3+ KB


Vemos que no se pudo extraer 14 registros, veamos el por qué.

In [221]:
steam_games.loc[(steam_games.release_year.isna()) & (steam_games.release_year_2.isna()) & (steam_games.release_date.notna())]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year,release_year_2
11836,LEVER Software,"['Indie', 'RPG']",The Dungeon of Destiny,The Dungeon of Destiny,http://store.steampowered.com/app/555980/The_D...,TBD,"['RPG', 'Indie', 'Dungeon Crawler', 'Great Sou...",http://steamcommunity.com/app/555980/reviews/?...,"['Single-player', 'Steam Achievements', 'Capti...",NaN,False,555980.0,LEVER Software,NaN,None
12540,Siberian Digital,"['Adventure', 'Indie', 'RPG', 'Simulation', 'S...",Hard Era: The Fantasy Defence,Hard Era: The Fantasy Defence,http://store.steampowered.com/app/692260/Hard_...,TBD,"['Simulation', 'Strategy', 'Adventure', 'RPG',...",http://steamcommunity.com/app/692260/reviews/?...,['Single-player'],NaN,False,692260.0,Siberian Digital,NaN,None
12751,Dagestan Technology,"['Adventure', 'Casual', 'Indie', 'Strategy']",Fantasy World,Fantasy World,http://store.steampowered.com/app/708790/Fanta...,Soon,"['Adventure', 'Indie', 'Casual', 'Strategy']",http://steamcommunity.com/app/708790/reviews/?...,"['Single-player', 'Multi-player', 'Local Multi...",NaN,False,708790.0,LaserStone,NaN,None
12936,Enjoy Games,"['Adventure', 'Indie']",Lepur,Lepur,http://store.steampowered.com/app/767600/Lepur/,Soon,"['Adventure', 'Indie']",http://steamcommunity.com/app/767600/reviews/?...,['Single-player'],NaN,False,767600.0,"Sochnoslivka, Luis Zuno",NaN,None
13034,Dagestan Technology,"['Adventure', 'Casual', 'Indie']",Franky the Bumwalker,Franky the Bumwalker,http://store.steampowered.com/app/761310/Frank...,SOON,"['Casual', 'Adventure', 'Indie']",http://steamcommunity.com/app/761310/reviews/?...,['Single-player'],0.99,False,761310.0,Onlyjoy`s production,NaN,None
13103,Dagestan Technology,"['Action', 'Adventure', 'Indie']",Howl,Howl,http://store.steampowered.com/app/513640/Howl/,Soon,"['Adventure', 'Indie', 'Action']",http://steamcommunity.com/app/513640/reviews/?...,['Single-player'],NaN,False,513640.0,Gunsquad,NaN,None
13244,Dagestan Technology,"['Adventure', 'Casual', 'Indie']",Mad Quad,Mad Quad,http://store.steampowered.com/app/760450/Mad_Q...,SOON,"['Adventure', 'Indie', 'Casual']",http://steamcommunity.com/app/760450/reviews/?...,['Single-player'],NaN,False,760450.0,Retardia Games,NaN,None
13246,Dagestan Technology,"['Action', 'Adventure', 'Casual', 'Indie']",Dungeon Escapist,Dungeon Escapist,http://store.steampowered.com/app/760440/Dunge...,SOON,"['Action', 'Adventure', 'Indie', 'Casual']",http://steamcommunity.com/app/760440/reviews/?...,['Single-player'],NaN,False,760440.0,Retardia Games,NaN,None
13425,Atriagames,"['Action', 'Indie']",FYD,FYD,http://store.steampowered.com/app/760700/FYD/,Soon,"['Action', 'Indie']",http://steamcommunity.com/app/760700/reviews/?...,['Single-player'],NaN,False,760700.0,Atriagames,NaN,None
13614,u̷n̸k̴n̶o̴w̶n̵,"['Action', 'Adventure', 'Indie', 'Simulation',...",DONT PLAY THIS GAME.EXE,DONT PLAY THIS GAME.EXE,http://store.steampowered.com/app/679240/DONT_...,0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶...,"['Early Access', 'Violent', 'Action', 'Simulat...",http://steamcommunity.com/app/679240/reviews/?...,['Single-player'],NaN,True,679240.0,u̷n̸k̴n̶o̴w̶n̵,NaN,None


In [222]:
steam_games.iloc[11836]

publisher                                            LEVER Software
genres                                             ['Indie', 'RPG']
app_name                                     The Dungeon of Destiny
title                                        The Dungeon of Destiny
url               http://store.steampowered.com/app/555980/The_D...
release_date                                                    TBD
tags              ['RPG', 'Indie', 'Dungeon Crawler', 'Great Sou...
reviews_url       http://steamcommunity.com/app/555980/reviews/?...
specs             ['Single-player', 'Steam Achievements', 'Capti...
price                                                           NaN
early_access                                                  False
id                                                         555980.0
developer                                            LEVER Software
release_year                                                    NaN
release_year_2                                  

Investigamos que al ingresar a la url de los registros que no pudimos extraer la fecha de lanzamiento, nos redirige directamente a la tienda principal de Steam, por ende, no es posible extraer los datos de la fecha de lanzamiento.

Ahora, de esos nuevos registros que extrajimos, vamos a sacar solo los valores del anio.

In [265]:
steam_games['release_year2'] = steam_games['release_year_2'].str.extract(r'(\b\d{4,})')

In [267]:
steam_games[['release_year_2','release_year2']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   release_year_2  90 non-null     object
 1   release_year2   64 non-null     object
dtypes: object(2)
memory usage: 502.2+ KB


Notamos que no nos da la misma cantidad, investiguemos un poco.

In [268]:
steam_games.loc[(steam_games.release_year_2.notna()) & (steam_games.release_year2.isna())]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year,release_year_2,release_year2
8338,Steven Colling,"['Indie', 'Simulation']",Orcish Inn,Orcish Inn,http://store.steampowered.com/app/533800/Orcis...,When it's done,"['Simulation', 'Indie', 'Survival']",http://steamcommunity.com/app/533800/reviews/?...,['Single-player'],NaN,False,533800.0,Steven Colling,NaN,To be announced,NaN
9335,Wisp Entertainment,"['Action', 'Adventure', 'Indie']",Attack On Kitten,Attack On Kitten,http://store.steampowered.com/app/618560/Attac...,TBA,"['Action', 'Indie', 'Adventure', 'Platformer']",http://steamcommunity.com/app/618560/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",NaN,False,618560.0,Wisp Entertainment,NaN,Coming soon,NaN
10275,deeparrowstudio,"['Free to Play', 'RPG', 'Strategy']",Samurai Forge,Samurai Forge,http://store.steampowered.com/app/617350/Samur...,soon,"['Free to Play', 'Strategy', 'RPG']",http://steamcommunity.com/app/617350/reviews/?...,"['Single-player', 'Online Multi-Player', 'Onli...",NaN,False,617350.0,deeparrowstudio,NaN,Coming soon,NaN
10483,Grotta Game Studio,['Indie'],Edge of Insanity,Edge of Insanity,http://store.steampowered.com/app/658340/Edge_...,14 July,"['Violent', 'Indie', 'Psychological Horror', '...",http://steamcommunity.com/app/658340/reviews/?...,['Single-player'],NaN,False,658340.0,Grotta Game Studio,NaN,Coming soon,NaN
10994,Space Pupper Games,"['Adventure', 'Casual', 'Indie']",Dungeon; Friends Escape!,Dungeon; Friends Escape!,http://store.steampowered.com/app/658280/Dunge...,Coming Soon,"['Adventure', 'Indie', 'Casual']",http://steamcommunity.com/app/658280/reviews/?...,"['Single-player', 'Online Multi-Player', 'Onli...",NaN,False,658280.0,Space Pupper Games,NaN,Coming soon,NaN
12996,NitorInc. Collab,"['Action', 'Casual']",NitorInc.: Touhou Microgames!,NitorInc.: Touhou Microgames!,http://store.steampowered.com/app/772260/Nitor...,Please wait warmly,"['Casual', 'Action', 'Singleplayer', 'Female P...",http://steamcommunity.com/app/772260/reviews/?...,['Single-player'],NaN,False,772260.0,NitorInc. Collab,NaN,Coming soon,NaN
13008,gillenew,"['Action', 'Indie', 'RPG', 'Early Access']",Sword of Fireheart - The Awakening Element,Sword of Fireheart - The Awakening Element,http://store.steampowered.com/app/503490/Sword...,early access,"['Early Access', 'Action', 'RPG', 'Indie', 'Pi...",http://steamcommunity.com/app/503490/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",NaN,True,503490.0,"gillenew,Fraccaro",NaN,Coming soon,NaN
13307,Headup Games,"['Indie', 'Simulation', 'Strategy', 'Early Acc...",Space Battle Core,Space Battle Core,http://store.steampowered.com/app/437010/Space...,Coming Soon,"['Early Access', 'Strategy', 'Simulation', 'In...",http://steamcommunity.com/app/437010/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",NaN,True,437010.0,Korion,NaN,Coming soon,NaN
13595,Oak Gear,"['Action', 'Adventure', 'Free to Play']",Lost Squad,Lost Squad,http://store.steampowered.com/app/284080/Lost_...,Coming soon,"['Free to Play', 'Action', 'Adventure', 'FPS',...",http://steamcommunity.com/app/284080/reviews/?...,"['Single-player', 'Co-op', 'Steam Achievements...",NaN,False,284080.0,Oak Gear,NaN,Coming soon,NaN
13646,Dotware,"['Indie', 'Strategy']",Akhenaten: Rule as Pharaoh,Akhenaten: Rule as Pharaoh,http://store.steampowered.com/app/719160/Akhen...,Coming Soon,"['Strategy', 'Indie']",http://steamcommunity.com/app/719160/reviews/?...,['Single-player'],NaN,False,719160.0,Dotware,NaN,Coming soon,NaN


Vemos que no tenemos los datos del anio del lanzamiento en estos registros, asi que obviaremos esto. <p>
Ahora uniremos la columna de *'release_year'* con *'release_year2'*

In [269]:
steam_games['release_year'].fillna(steam_games['release_year2'], inplace=True)

Y procederemos a eliminar las columnas que creamos anteriormente para extraer el anio de lanzamiento.

In [272]:
steam_games.drop(columns = ['release_year_2','release_year2'], inplace = True)

In [273]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  bool   
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
 13  release_year  30028 non-null  object 
dtypes: bool(1), float64(1), object(12)
memory usage: 3.2+ MB


Ahora eliminaremos las filas que contengan valores faltante en todas las columnas

In [66]:
steam_games.dropna(how='all',inplace=True)

Verificamos

In [67]:
steam_games.shape

(32135, 15)

Veamos el porcentaje de valores vacios en cada columna

In [277]:
steam_games.isna().mean()

publisher       0.251159
genres          0.102163
app_name        0.000062
title           0.063793
url             0.000000
release_date    0.064322
tags            0.005072
reviews_url     0.000062
specs           0.020850
price           0.042850
early_access    0.000000
id              0.000062
developer       0.102661
release_year    0.065567
dtype: float64

Notamos que la columna *'discount_price'* tiene 99% de valores vacios y *'metascore'* tiene casi 92%. Asi que eliminaremos estas columnas tambien, ya que ademas son irrelevantes para nuestras consultas.

In [35]:
steam_games.drop(columns=['discount_price','metascore'],inplace=True)

In [278]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  bool   
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
 13  release_year  30028 non-null  object 
dtypes: bool(1), float64(1), object(12)
memory usage: 3.2+ MB


Vemos que en la columna *'id'* hay 2 registros vacios, investiguemos esos registros.

In [13]:
steam_games.loc[steam_games['id'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year


A simple vista, observamos que uno de esos registros tiene la mayoria de las columnas vacias, y en el otro registro podemos obtener el id de ese item con la url que nos brinda este dataset.
Asi que primero eliminaremos esa fila donde tiene la mayoria de los datos vacios y luego vamos a corroborar si el id de la fila 30961 es igual al item_id que tenemos en el DataFrame **items**.

Eliminamos la fila con el índice 74.

In [ ]:
steam_games.drop(index = 74,inplace = True)

Ahora buscamos en el DataFrame **items** los registros donde contengan el nombre del item del DataFrame **steam_games**

In [6]:
items[items['item_name'].str.find("Batman: Arkham City") != -1]

,item_id,item_name,user_id,genres,playtime_forever,playtime_2weeks
287,200260,Batman: Arkham City GOTY,76561197970982479,Action,100,0
288,200260,Batman: Arkham City GOTY,76561197970982479,Adventure,100,0
836,200260,Batman: Arkham City GOTY,js41637,Action,1066,0
837,200260,Batman: Arkham City GOTY,js41637,Adventure,1066,0
2858,57400,Batman: Arkham City™,Riot-Punch,NaN,959,0
...,...,...,...,...,...,...
10802120,200260,Batman: Arkham City GOTY,76561198227671916,Adventure,144,0
10802853,200260,Batman: Arkham City GOTY,Alexexe,Action,192,0
10802854,200260,Batman: Arkham City GOTY,Alexexe,Adventure,192,0
10808836,200260,Batman: Arkham City GOTY,IShipHarmony,Action,0,0


Bien, con esto podemos afirmar que el id que nos brindaba la url del DataFrame **steam_games** es igual al item_id del DataFrame **items**. Asi que imputaremos este dato.

In [20]:
steam_games.loc[30961,'id'] = 200260

200260.0

In [19]:
steam_games.iloc[30960]

publisher       Warner Bros. Interactive Entertainment, Feral ...
genres                                    ['Action', 'Adventure']
app_name           Batman: Arkham City - Game of the Year Edition
title              Batman: Arkham City - Game of the Year Edition
url                      http://store.steampowered.com/app/200260
release_date                                           2012-09-07
tags            ['Action', 'Open World', 'Batman', 'Adventure'...
reviews_url                                                   NaN
specs           ['Single-player', 'Steam Achievements', 'Steam...
price                                                       19.99
early_access                                                False
id                                                       200260.0
developer              Rocksteady Studios,Feral Interactive (Mac)
release_year                                               2012.0
Name: 30961, dtype: object

Analicemos la columna *'price'*<p>
Veamos los valores unicos sin contar los nulos.

In [60]:
steam_games['price'][steam_games['price'].notnull()].unique()

array(['4.99', 'Free To Play', 'Free to Play', '0.99', '2.99', '3.99',
       '9.99', '18.99', '29.99', 'Free', '10.99', '1.59', '14.99', '1.99',
       '59.99', '8.99', '6.99', '7.99', '39.99', '19.99', '7.49', '12.99',
       '5.99', '2.49', '15.99', '1.25', '24.99', '17.99', '61.99', '3.49',
       '11.99', '13.99', 'Free Demo', 'Play for Free!', '34.99', '74.76',
       '1.49', '32.99', '99.99', '14.95', '69.99', '16.99', '79.99',
       '49.99', '5.0', '44.99', '13.98', '29.96', '119.99', '109.99',
       '149.99', '771.71', 'Install Now', '21.99', '89.99',
       'Play WARMACHINE: Tactics Demo', '0.98', '139.92', '4.29', '64.99',
       'Free Mod', '54.99', '74.99', 'Install Theme', '0.89',
       'Third-party', '0.5', 'Play Now', '299.99', '1.29', '3.0', '15.0',
       '5.49', '23.99', '49.0', '20.99', '10.93', '1.39',
       'Free HITMAN™ Holiday Pack', '36.99', '4.49', '2.0', '4.0', '9.0',
       '234.99', '1.95', '1.5', '199.0', '189.0', '6.66', '27.99',
       '10.49', '129.

Cambiamos los valores NaN por **'--'**

In [61]:
steam_games['price'] = steam_games['price'].fillna('--')

Veamos los valores que no son dígitos en la columna *'price'*

In [62]:
patron = r'^\D*$' # Definimos la expresión regular para buscar valores sin dígitos

lista = steam_games[steam_games['price'].str.contains(patron, case=False, regex=True)]['price'].unique()

In [63]:
lista

array(['Free To Play', 'Free to Play', '--', 'Free', 'Free Demo',
       'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo',
       'Free Mod', 'Install Theme', 'Third-party', 'Play Now',
       'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try',
       'Free Movie', 'Free to Use'], dtype=object)

Analizando esta lista, entendemos que los registros que tengan estos valores son Free. Asi que lo modificaremos, pero antes, excluiremos de la lista el valor '--'.

In [64]:
lista = [valor for valor in lista if valor != '--']

In [65]:
lista

['Free To Play',
 'Free to Play',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Free to Try',
 'Free Movie',
 'Free to Use']

In [66]:
steam_games['price'] = steam_games['price'].replace(lista, '0.0')

In [74]:
steam_games['price'][steam_games['price'].notnull()].unique()

array(['4.99', '0.0', '0.99', '2.99', '3.99', '9.99', '18.99', '29.99',
       '--', '10.99', '1.59', '14.99', '1.99', '59.99', '8.99', '6.99',
       '7.99', '39.99', '19.99', '7.49', '12.99', '5.99', '2.49', '15.99',
       '1.25', '24.99', '17.99', '61.99', '3.49', '11.99', '13.99',
       '34.99', '74.76', '1.49', '32.99', '99.99', '14.95', '69.99',
       '16.99', '79.99', '49.99', '5.0', '44.99', '13.98', '29.96',
       '119.99', '109.99', '149.99', '771.71', '21.99', '89.99', '0.98',
       '139.92', '4.29', '64.99', '54.99', '74.99', '0.89', '0.5',
       '299.99', '1.29', '3.0', '15.0', '5.49', '23.99', '49.0', '20.99',
       '10.93', '1.39', '36.99', '4.49', '2.0', '4.0', '9.0', '234.99',
       '1.95', '1.5', '199.0', '189.0', '6.66', '27.99', '10.49',
       '129.99', '179.0', '26.99', '399.99', '31.99', '399.0', '20.0',
       '40.0', '3.33', '199.99', '22.99', '320.0', '38.85', '71.7',
       '59.95', '995.0', '27.49', '3.39', '6.0', '19.95', '499.99',
       '16.06', '

Notamos que hay valores donde especifican el precio pero tiene texto, asi que eliminaremos el texto y solo dejaremos el valor del precio.

In [68]:
steam_games['price'] = steam_games['price'].replace('Starting at $499.00', '499.00')

Eliminamos la columna *''reviews_url* ya que es irrelevante.

In [38]:
steam_games.drop(columns='reviews_url', inplace= True)

Verificamos los duplicados segun el id del item

In [43]:
steam_games.duplicated(subset='id').sum()

0

Eliminamos los duplicados segun el id

In [42]:
steam_games.drop_duplicates(subset= 'id', inplace= True)

Guardaremos en un nuevo archivo CSV este DataFrame

In [44]:
steam_games.to_csv('./dataset/Modificados/steam_games.csv', encoding= 'utf-8', index=False)

#### DataFrame 'reviews'

En este DataFrame vamos a convertir en formato fecha *YY-MM-DD* los valores de la columna **'posted'** para facilitar el analisis y consultas.

In [4]:
reviews.head()

,user_id,item_id,posted,last_edited,funny,recommend,review,helpful
0,76561197970982479,1250,"Posted November 5, 2011.",NaN,NaN,True,Simple yet with great replayability. In my opi...,No ratings yet
1,76561197970982479,22200,"Posted July 15, 2011.",NaN,NaN,True,It's unique and worth a playthrough.,No ratings yet
2,76561197970982479,43110,"Posted April 21, 2011.",NaN,NaN,True,Great atmosphere. The gunplay can be a bit chu...,No ratings yet
3,js41637,251610,"Posted June 24, 2014.",NaN,NaN,True,I know what you think when you see this title ...,15 of 20 people (75%) found this review helpful
4,js41637,227300,"Posted September 8, 2013.",NaN,NaN,True,For a simple (it's actually not all that simpl...,0 of 1 people (0%) found this review helpful


In [5]:
reviews['posted'][0]

'Posted November 5, 2011.'

Vemos que el mes esta escrito en formato texto y en ingles, ademas, hay una palabra al inicio de cada registro, asi que procederemos a eliminar esa palabra y cambiar el formato de las fechas.

* Eliminamos la palabra 'Posted' de los registros

In [14]:
reviews['posted'] = reviews['posted'].replace('Posted ', '',regex=True)

In [15]:
reviews['posted'][0]

'November 5, 2011.'

* Cambiamos el formato de las fechas.

In [16]:
reviews['posted'] = pd.to_datetime(reviews['posted'], errors='coerce')

In [17]:
reviews['posted'][0]

Timestamp('2011-11-05 00:00:00')

In [18]:
reviews.head()

,user_id,item_id,posted,last_edited,funny,recommend,review,helpful
0,76561197970982479,1250,2011-11-05,NaN,NaN,True,Simple yet with great replayability. In my opi...,No ratings yet
1,76561197970982479,22200,2011-07-15,NaN,NaN,True,It's unique and worth a playthrough.,No ratings yet
2,76561197970982479,43110,2011-04-21,NaN,NaN,True,Great atmosphere. The gunplay can be a bit chu...,No ratings yet
3,js41637,251610,2014-06-24,NaN,NaN,True,I know what you think when you see this title ...,15 of 20 people (75%) found this review helpful
4,js41637,227300,2013-09-08,NaN,NaN,True,For a simple (it's actually not all that simpl...,0 of 1 people (0%) found this review helpful


In [20]:
reviews.to_csv('./dataset/Modificados/reviews.csv', encoding= 'utf-8', index=False)

In [77]:
#Feature Engineers

In [34]:
##pruebas

In [3]:
steam_games = pd.read_csv('./dataset/Modificados/steam_games.csv')

In [2]:
items = pd.read_csv('./dataset/Modificados/items.csv')

In [4]:
users = pd.read_csv('./dataset/Modificados/users.csv')

In [5]:
reviews = pd.read_csv('./dataset/Modificados/reviews.csv')